In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/drive/MyDrive/MACHINE LEARNING LAB/Dataset/sentimentdataset.csv")

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  732 non-null    int64  
 1   Unnamed: 0    732 non-null    int64  
 2   Text          732 non-null    object 
 3   Sentiment     732 non-null    object 
 4   Timestamp     732 non-null    object 
 5   User          732 non-null    object 
 6   Platform      732 non-null    object 
 7   Hashtags      732 non-null    object 
 8   Retweets      732 non-null    float64
 9   Likes         732 non-null    float64
 10  Country       732 non-null    object 
 11  Year          732 non-null    int64  
 12  Month         732 non-null    int64  
 13  Day           732 non-null    int64  
 14  Hour          732 non-null    int64  
dtypes: float64(2), int64(6), object(7)
memory usage: 85.9+ KB


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [6]:
df = df[['Text', 'Sentiment']]

In [7]:
df['Sentiment'] = df['Sentiment'].factorize()[0]  # Convert sentiment to numerical

In [8]:
df['Sentiment']

,Sentiment
0,0
1,1
2,0
3,0
4,2
...,...
727,278
728,278
729,278
730,278


In [9]:
tfidf = TfidfVectorizer(max_features=5000)       # Limit features for performance
X = tfidf.fit_transform(df['Text']).toarray()    # Convert text to TF-IDF features
y = df['Sentiment']                              # Target variable

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Track and plot accuracy over epochs
train_acc = []
for i, pred in enumerate(rf.staged_predict(X_train)):
    acc = accuracy_score(y_train, pred)
    train_acc.append(acc)

# Plot training accuracy
plt.plot(train_acc, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy per Epoch')
plt.legend()
plt.show()

AttributeError: 'RandomForestClassifier' object has no attribute 'staged_predict'

In [12]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [13]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model from Grid Search
best_rf = grid_search.best_estimator_

# Step 6: Make predictions and evaluate
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Define the hyperparameter grid
n_estimators_options = [100, 200, 300]
max_depth_options = [10, 20, None]
min_samples_split_options = [2, 5, 10]
min_samples_leaf_options = [1, 2, 4]

# Store results
results = []

# Manually loop through hyperparameters
for n_estimators in n_estimators_options:
    for max_depth in max_depth_options:
        for min_samples_split in min_samples_split_options:
            for min_samples_leaf in min_samples_leaf_options:
                # Initialize and train model
                rf = RandomForestClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf,
                    random_state=42
                )
                rf.fit(X_train, y_train)

                # Predict and evaluate
                y_pred = rf.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)

                # Save results
                results.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
                    'accuracy': accuracy
                })
                print(f"n_estimators: {n_estimators}, max_depth: {max_depth}, min_samples_split: {min_samples_split}, "
                      f"min_samples_leaf: {min_samples_leaf} => Accuracy: {accuracy}")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Plot accuracy for each combination
plt.figure(figsize=(12, 8))
for n_estimators in n_estimators_options:
    subset = results_df[results_df['n_estimators'] == n_estimators]
    plt.plot(np.arange(len(subset)), subset['accuracy'], label=f'n_estimators={n_estimators}')

plt.title('Accuracy for Different Hyperparameter Combinations')
plt.xlabel('Combination Index')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()